In [17]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import os

In [18]:
(x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train_full = x_train_full.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train_full = np.expand_dims(x_train_full, -1)
x_test = np.expand_dims(x_test, -1)

In [20]:
num_train_samples = 20000
x_train_small = x_train_full[:num_train_samples]
y_train_small = y_train_full[:num_train_samples]
print(f"Данных для обучения взято: {x_train_small.shape[0]}")

Данных для обучения взято: 20000


In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print("Начинаем обучение на урезанной выборке (20,000)...")
model.fit(x_train_small, y_train_small, epochs=5, validation_data=(x_test, y_test))
print("Обучение битти.")

Начинаем обучение на урезанной выборке (20,000)...
Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - accuracy: 0.8238 - loss: 0.5827 - val_accuracy: 0.9568 - val_loss: 0.1417
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.9692 - loss: 0.1081 - val_accuracy: 0.9716 - val_loss: 0.0890
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.9824 - loss: 0.0635 - val_accuracy: 0.9727 - val_loss: 0.0849
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 23s 37ms/step - accuracy: 0.9880 - loss: 0.0429 - val_accuracy: 0.9778 - val_loss: 0.0715
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 32s 22ms/step - accuracy: 0.9905 - loss: 0.0332 - val_accuracy: 0.9772 - val_loss: 0.0706
Обучение битти.


In [23]:
print("Оценка производительности модели:")
y_pred_test_probs = model.predict(x_test)
y_pred_test = np.argmax(y_pred_test_probs, axis=1)
target_names = [f"Цифра {i}" for i in range(10)]
print(classification_report(y_test, y_pred_test, target_names=target_names))

Оценка производительности модели:
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
              precision    recall  f1-score   support

     Цифра 0       0.98      0.99      0.98       980
     Цифра 1       0.99      0.99      0.99      1135
     Цифра 2       0.96      0.98      0.97      1032
     Цифра 3       0.98      0.98      0.98      1010
     Цифра 4       0.98      0.98      0.98       982
     Цифра 5       0.99      0.96      0.97       892
     Цифра 6       0.98      0.98      0.98       958
     Цифра 7       0.97      0.97      0.97      1028
     Цифра 8       0.98      0.96      0.97       974
     Цифра 9       0.96      0.98      0.97      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



In [25]:
print("Инференс (поиск ошибок) на ВСЕХ данных")
x_all = np.concatenate([x_train_full, x_test], axis=0)
y_all = np.concatenate([y_train_full, y_test], axis=0)
print(f"Общее количество изображений для инференса: {x_all.shape[0]}")
all_predictions_probs = model.predict(x_all)
all_predictions = np.argmax(all_predictions_probs, axis=1)
misclassified_indices = np.where(all_predictions != y_all)[0]
fingerprint_images = x_all[misclassified_indices]
fingerprint_labels = y_all[misclassified_indices]
print("\n--- РЕЗУЛЬТАТ ---")
print(f"Найдено {len(fingerprint_images)} плохо распознаваемых изображений.")
print(f"Итоговый размер 'Fingerprint Dataset': {fingerprint_images.shape}")

Инференс (поиск ошибок) на ВСЕХ данных
Общее количество изображений для инференса: 70000
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step

--- РЕЗУЛЬТАТ ---
Найдено 1304 плохо распознаваемых изображений.
Итоговый размер 'Fingerprint Dataset': (1304, 28, 28, 1)


In [26]:
output_dir = 'fingerprint_dataset'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Создана папка: {output_dir}")
print(f"Сохранение {len(fingerprint_images)} изображений в папку...")

Создана папка: fingerprint_dataset
Сохранение 1304 изображений в папку...


In [27]:
for i, image_index in enumerate(misclassified_indices):
    image_array = x_all[image_index]
    true_label = y_all[image_index]
    predicted_label = all_predictions[image_index]

    filename = f"{str(i).zfill(4)}__pred-{predicted_label}__true-{true_label}.png"
    filepath = os.path.join(output_dir, filename)

    plt.imsave(filepath, np.squeeze(image_array), cmap='gray')